In [12]:
import requests
import pandas as pd

In [32]:
# URL de l'API pour le classement Ligue 1 2024–2025
url = "https://www.thesportsdb.com/api/v1/json/3/lookuptable.php?l=4334&s=2025-2026"

# Requête à l'API
response = requests.get(url)
r = response.json()

# Vérification de la clé 'table'
if 'table' not in r:
    print("❌ Données de classement introuvables.")
else:
    table = r['table']

In [23]:
# Construction du DataFrame
data = pd.DataFrame([{
    "Pos": team["intRank"],
    "Team": team["strTeam"],
    "M": team["intPlayed"],
    "W": team["intWin"],
    "D": team["intDraw"],
    "L": team["intLoss"],
    "G": team["intGoalsFor"],
    "GA": team["intGoalsAgainst"],
    "Diff": team["intGoalDifference"],
    "PTS": team["intPoints"]
} for team in table])

# Tri par points (décroissant)
data = data.sort_values(by="PTS", ascending=False).reset_index(drop=True)

# Supprimer l'ancienne colonne Pos
if "Pos" in data.columns:
    data = data.drop(columns=["Pos"])

# Recalcul du rang
data.insert(0, "Pos", data.index + 1)

# Affichage
print("🏆 Classement Ligue 1 – 2025/2026 :")
display(data)

🏆 Classement Ligue 1 – 2025/2026 :


,Pos,Team,M,W,D,L,G,GA,Diff,PTS
0,1,Paris SG,3,3,0,0,8,3,5,9
1,2,Lyon,3,3,0,0,5,0,5,9
2,3,Lille,3,2,1,0,11,4,7,7
3,4,Monaco,3,2,0,1,6,4,2,6
4,5,Lens,3,2,0,1,5,3,2,6


In [24]:
def get_team_info(team_name, data):
    team = data[data["Team"] == team_name]
    if team.empty:
        raise ValueError(f"⚠️ L'équipe '{team_name}' n'existe pas dans le classement. "
                         f"Noms disponibles : {list(data['Team'].unique())}")
    return team.iloc[0]

def generate_odds(teamA_name, teamB_name, data):
    """
    Génère les cotes 1X2 pour un match entre teamA et teamB.
    
    teamA_name, teamB_name : noms des équipes (chaînes)
    data : DataFrame du classement (doit contenir 'Team', 'PTS', 'Diff')
    """
    # Extraction des infos depuis le DataFrame
    teamA = get_team_info(teamA_name, data)
    teamB = get_team_info(teamB_name, data)

    # Score de force basé sur les points et la différence de buts
    strengthA = int(teamA["PTS"]) + int(teamA["Diff"]) * 0.3
    strengthB = int(teamB["PTS"]) + int(teamB["Diff"]) * 0.3

    # Probabilités brutes
    pA = strengthA / (strengthA + strengthB)
    pB = strengthB / (strengthA + strengthB)
    pDraw = 0.15 + 0.1 * (1 - abs(pA - pB))  # nul plus probable si forces proches

    # Normalisation
    total = pA + pB + pDraw
    pA /= total
    pB /= total
    pDraw /= total

    # Conversion en cotes décimales (1/probabilité) + marge bookmaker
    margin = 1.05
    oddsA = round((1 / pA) * margin, 2)
    oddsDraw = round((1 / pDraw) * margin, 2)
    oddsB = round((1 / pB) * margin, 2)

    return {
        "Match": f"{teamA_name} vs {teamB_name}",
        "1": oddsA,
        "X": oddsDraw,
        "2": oddsB
    }

In [25]:
season_data = pd.DataFrame([{
    "Team": team["strTeam"],
    "M": int(team["intPlayed"]),
    "W": int(team["intWin"]),
    "D": int(team["intDraw"]),
    "L": int(team["intLoss"]),
    "G": int(team["intGoalsFor"]),
    "GA": int(team["intGoalsAgainst"]),
    "Diff": int(team["intGoalDifference"]),
    "PTS": int(team["intPoints"])
} for team in table])
try:
    home_team = "Rennes"
    away_team = "Lyon"

    odds = generate_odds(home_team, away_team, season_data)

    print(f"\n📊 Predicted Odds: {home_team} vs {away_team}")
    for k, v in odds.items():
        print(f"  {k}: {v}")

except Exception as e:
    print("Error:", e)

Error: ⚠️ L'équipe 'Rennes' n'existe pas dans le classement. Noms disponibles : ['Paris SG', 'Lyon', 'Lille', 'Monaco', 'Lens']
